In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/retailtransactiondata/Retail_Data_Response.csv
/kaggle/input/retailtransactiondata/Retail_Data_Transactions.csv


In [2]:
df = pd.read_csv('/kaggle/input/retailtransactiondata/Retail_Data_Transactions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  125000 non-null  object
 1   trans_date   125000 non-null  object
 2   tran_amount  125000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.9+ MB


In [3]:
print('Null Summary :') 
print(df.isnull().sum())

Null Summary :
customer_id    0
trans_date     0
tran_amount    0
dtype: int64


In [4]:
import datetime as dt

def create_train_test(df) :
    #Removing any duplicate from dataframe
    df = df[~df.duplicated()]
    
    #Changing data type into datetime
    df['trans_date'] = pd.to_datetime(df['trans_date'])
    
    #Generating new columns from our datetime dataset
    df['year'] = df['trans_date'].dt.year
    df['month'] = df['trans_date'].dt.month
    df['dayofyear'] = df['trans_date'].dt.dayofyear
    df['dayofmonth'] = df['trans_date'].dt.day
    df['dayofweek'] = df['trans_date'].dt.dayofweek
    df['weekofyear'] = df['trans_date'].dt.weekofyear
    
    #Setting datetime data as the df index
    df = df.set_index('trans_date')
    
    #Splitting into features and target
    X,y = df[['year','month','dayofyear','dayofmonth','dayofweek','weekofyear']], df['tran_amount']
    
    #Return the final split
    return X,y

In [5]:
X,y = create_train_test(df)
print(X)

            year  month  dayofyear  dayofmonth  dayofweek  weekofyear
trans_date                                                           
2013-02-11  2013      2         42          11          0           7
2015-03-15  2015      3         74          15          6          11
2013-02-26  2013      2         57          26          1           9
2011-11-16  2011     11        320          16          2          46
2013-11-20  2013     11        324          20          2          47
...          ...    ...        ...         ...        ...         ...
2011-06-26  2011      6        177          26          6          25
2014-08-19  2014      8        231          19          1          34
2014-11-28  2014     11        332          28          4          48
2013-12-14  2013     12        348          14          5          50
2014-12-13  2014     12        347          13          5          50

[124994 rows x 6 columns]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [6]:
X = X.sort_index(ascending=True)
X

,year,month,dayofyear,dayofmonth,dayofweek,weekofyear
trans_date,,,,,,
2011-05-16,2011,5,136,16,0,20
2011-05-16,2011,5,136,16,0,20
2011-05-16,2011,5,136,16,0,20
2011-05-16,2011,5,136,16,0,20
2011-05-16,2011,5,136,16,0,20
...,...,...,...,...,...,...
2015-03-16,2015,3,75,16,0,12
2015-03-16,2015,3,75,16,0,12
2015-03-16,2015,3,75,16,0,12


In [7]:
y = y.sort_index(ascending=True)
y

trans_date
2011-05-16     90
2011-05-16    101
2011-05-16     63
2011-05-16     84
2011-05-16     79
             ... 
2015-03-16     31
2015-03-16     66
2015-03-16     80
2015-03-16    104
2015-03-16     46
Name: tran_amount, Length: 124994, dtype: int64

In [8]:
X_train = X[:int(0.8*len(X))]
X_test = X[int(0.8*len(X)):]
y_train = y[:int(0.8*len(y))]
y_test = y[int(0.8*len(y)):]

print(X_train)

            year  month  dayofyear  dayofmonth  dayofweek  weekofyear
trans_date                                                           
2011-05-16  2011      5        136          16          0          20
2011-05-16  2011      5        136          16          0          20
2011-05-16  2011      5        136          16          0          20
2011-05-16  2011      5        136          16          0          20
2011-05-16  2011      5        136          16          0          20
...          ...    ...        ...         ...        ...         ...
2014-06-08  2014      6        159           8          6          23
2014-06-08  2014      6        159           8          6          23
2014-06-08  2014      6        159           8          6          23
2014-06-08  2014      6        159           8          6          23
2014-06-08  2014      6        159           8          6          23

[99995 rows x 6 columns]


In [9]:
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#Set parameters for our XGBoost Model
params = {
    'n_estimators':[500],
    'min_child_weight':[4,5], 
    'gamma':[i/10.0 for i in range(3,6)],  
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(6,11)], 
    'max_depth': [2,3,4,6,7],
    'objective': ['reg:squarederror', 'reg:tweedie'],
    'booster': ['gblinear'],
    'eval_metric': ['rmse'],
    'eta': [i/10.0 for i in range(3,6)],
}

n_iter = 10
cv = 5
scoring = 'neg_mean_squared_error'

#Function for getting the best params on the model 
def xgb_model_best_estimator(X_train, X_test, y_train, y_test, params, n_iter, cv, scoring) :
    model = xgb.XGBRegressor(nthread = -1)
    
    clf = RandomizedSearchCV(model, param_distributions = params,
                            n_iter = n_iter, cv = cv, scoring = scoring)
    clf.fit(X_train,y_train, 
              eval_set=[(X_train,y_train),(X_test,y_test)],
             verbose = False)
    
    best_parameter = clf.best_estimator_
    
    return best_parameter


In [10]:
best_param = xgb_model_best_estimator(X_train, X_test, y_train, y_test, params, n_iter, cv, scoring)
print(best_param)

[07:41:09] WARNING: ../src/learner.cc:627: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[07:41:17] WARNING: ../src/learner.cc:627: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[07:41:25] WARNING: ../src/learner.cc:627: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight", "subsample" } might not be used.

  

In [11]:
yhat = best_param.predict(X_test)

print(f'MAE : {mean_absolute_error(y_test,yhat)}')
print(f'MSE : {mean_squared_error(y_test,yhat)}')

MAE : 19.334745717099192
MSE : 528.0600528401017


In [12]:
df_y = y_test.reset_index()

df_y['predicted'] = yhat

df_y #Show Final Dataframe for comparison

,trans_date,tran_amount,predicted
0,2014-06-08,70,64.806076
1,2014-06-08,37,64.806076
2,2014-06-08,99,64.806076
3,2014-06-08,53,64.806076
4,2014-06-08,82,64.806076
...,...,...,...
24994,2015-03-16,31,64.957573
24995,2015-03-16,66,64.957573
24996,2015-03-16,80,64.957573
24997,2015-03-16,104,64.957573
